In [3]:
import pandas as pd
import numpy as np
import whisper

In [4]:
model = whisper.load_model("small")
result = model.transcribe("/workspaces/Gesture-Language-Alignment-in-Instructional-Videos/00_videos/EasyBlueberryMuffinsRecipe.mp4")
print(result["text"])

100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 77.7MiB/s]
/home/codespace/.python/current/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hey guys, I'm Dina from SimplyHomeCook.com and today I'll be showing how to make these fluffy and moist blueberry muffins. They're loaded with a ton of blueberries and they're topped the simple and delicious brown sugar streusel. So you want to start off by combining all of your dry ingredients. So in a large bowl combine two cups of all-purpose flour, a teaspoon of baking powder, half a teaspoon of baking soda, and a quarter teaspoon of salt. Then whisk it all together and reserve one tablespoon of that flour mixture. Now mix that reserved tablespoon of the flour mixture with one and a half cups of fresh blueberries. The reason why we add that little tiny bit of flour to those blueberries is when we add them to the batter, during baking the blueberries don't sink as fast when they're coated in flour. Now add three eggs into a large bowl and whisk them on high speed using an electric mixer. So you want to do this for about three minutes until the eggs become fluffy and pale. Then cont

In [ ]:
https://www.youtube.com/watch?v=rU675lZ1ynA&ab_channel=NYTCooking